## Laboratorio N°1
### Materia: Series de Tiempo
#### Alumnos: Díaz, Carolina ; Fabro, Juan

Modelos de predicción para llamadas telefónicas por minutos para líneas prepagas y pospagas

In [ ]:
# Packages
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [ ]:
# Plot settings
sns.set_context('notebook') 
sns.set_style('ticks')
red='#D62728'
blue='#1F77B4'
%matplotlib inline


In [ ]:
plt.rcParams['figure.figsize'] = [15, 10]

In [ ]:
from datetime import datetime
now=datetime.now()
now

In [ ]:
print(now)

In [ ]:
type(now)

In [ ]:
now.year, now.month, now.day

Fechas para series de tiempo

In [ ]:
data=pd.read_csv('./datasets/Telefonia_movil.csv')
data['Año_Trimestre'] = data["Año"].astype(str) + "Q" + data["Trimestre"].map(str)
data.set_index('Año_Trimestre')
#data.index=data.index.to_period(freq='Q')

data.head()

In [ ]:
data_trim= pd.to_datetime(data['Año_Trimestre'])
data['Año_Trimestre'] = pd.PeriodIndex(data_trim, freq='Q')
data[:10]

In [ ]:
data['Minutos pospago salientes (miles)'] = data['Minutos pospago salientes (miles)'].str.replace('.', '').astype(int)
data.index=data['Año_Trimestre']

In [ ]:
datat= pd.DataFrame(data, columns=['Minutos pospago salientes (miles)', 'Minutos prepago salientes (miles)', 'Total de minutos salientes (miles)'])
datat.dtypes

In [ ]:
datat.describe().round(2)

Times series plot

In [ ]:
datat.plot();
plt.show()

In [ ]:
datat['2013'].plot(figsize=(6,4));

In [ ]:
datat['2014'].plot(figsize=(6,4));

In [ ]:
datat['2015'].plot(figsize=(6,4));

In [ ]:
datat['2016'].plot(figsize=(6,4));

In [ ]:
datat['2017'].plot(figsize=(6,4));

In [ ]:
datat['2018'].plot(figsize=(6,4));

In [ ]:
datat['quarter'] = datat.index.quarter
datat['year'] = datat.index.year

So to compare periods by ploting, we use Seaborn line plot using hue

In [ ]:
sns.lineplot('quarter','Total de minutos salientes (miles)',hue='year',data=datat, markers='none');

In [ ]:
sns.lineplot('quarter','Minutos pospago salientes (miles)',hue='year',data=datat, markers='none');

In [ ]:
sns.lineplot('quarter','Minutos prepago salientes (miles)',hue='year',data=datat, markers='none');


Resumimos datos para todo el período bajo análisis en una línea y su intervalo


In [ ]:
sns.lineplot('year','Minutos pospago salientes (miles)',data=datat[['year','Minutos pospago salientes (miles)']]);

In [ ]:
sns.lineplot('year','Minutos prepago salientes (miles)',data=datat[['year','Minutos prepago salientes (miles)']]);

In [ ]:
datat.boxplot(column='Minutos pospago salientes (miles)',by='quarter');

la dispersión en los trimestres varía, aparentemente se incrementa en el segundo y cuarto trimestre pero siendo el úlitmo el que se despega con respecto a los demás

In [ ]:
datat.boxplot(column='Minutos pospago salientes (miles)',by='year');

In [ ]:
datat.boxplot(column='Minutos prepago salientes (miles)',by='quarter');

In [ ]:
datat.boxplot(column='Minutos prepago salientes (miles)',by='year');

Visualizamos la tendencia

In [ ]:
from pandas.plotting import lag_plot

In [ ]:
#We can also do a lag graph

lag_plot(datat['Minutos pospago salientes (miles)']);

In [ ]:
lag_plot(datat['Minutos pospago salientes (miles)'],lag=12,);

In [ ]:
plt.rcParams['figure.figsize'] = [20, 200]
for i in range(14):
   plt.subplot(14,2,i+1)
   plt.title("lag " + str(i+1))
   lag_plot(datat['Minutos pospago salientes (miles)'],lag=i+1);

### Random Walk

### Líneas Pospagas

In [ ]:
# ultimos valores de la serie original
datat= datat.drop(['quarter', 'year', 'Total de minutos salientes (miles)'], axis=1)
datat.tail()

Creamos un rango de fechas o períodos a partir del índice de tiempo que sigue a la última observación en los datos.


In [ ]:
h = 4 # cuatro trimestres posteriores son los que queremos predecir.
test=pd.period_range(start=datat.index[-1]+1, periods=h, freq='Q')
test


In [ ]:

pred_pospagas=pd.Series(np.repeat(datat['Minutos pospago salientes (miles)'].iloc[-1],h), index=test) # the forecast repeats the last observed values h times
pred_pospagas


Para construir los intervalos de la predicción obtenemos DS de los errores

In [ ]:
resid_pos=datat['Minutos pospago salientes (miles)']-datat['Minutos pospago salientes (miles)'].shift(1) # the shift lags the series by one period, trajo la serie para más adelante entonces restamos a la serie original la nueva serie
sigma_pos = resid_pos.std()# ver los si los residuos se distribuyen normal, graficar
round(sigma_pos,3)

In [ ]:
intv_pos = pd.concat([pred_pospagas-stats.norm.ppf(0.975)*sigma_pos*np.sqrt(np.arange(1,h+1)),# cortamos las colas de la normal, calculamos IC
                  
                   pred_pospagas+stats.norm.ppf(0.975)*sigma_pos*np.sqrt(np.arange(1,h+1))], axis=1)
intv_pos.round(3)

In [ ]:
from plotting import fanchart

point_pred = (intv_pos[0]+intv_pos[1])/2

fig, ax = fanchart(datat['Minutos pospago salientes (miles)'],point_pred,intv_pos,intv_pos,intv_pos)

plt.show()

Líneas prepagas

In [ ]:
pred_pre=pd.Series(np.repeat(datat['Minutos prepago salientes (miles)'].iloc[-1], h), index=test) # the forecast repeats the last observed values h times
pred_pre

In [ ]:
resid_pre=datat['Minutos prepago salientes (miles)']-datat['Minutos prepago salientes (miles)'].shift(1) # the shift lags the series by one period
sigma_pre = resid_pre.std()
round(sigma_pre,3)

In [ ]:
intv_pre = pd.concat([pred_pre-stats.norm.ppf(0.975)*sigma_pos*np.sqrt(np.arange(1,h+1)),
                   pred_pre+stats.norm.ppf(0.975)*sigma_pos*np.sqrt(np.arange(1,h+1))], axis=1)
intv_pre.round(3)

In [ ]:
point_pred_pre = (intv_pre[0]+intv_pre[1])/2

fig, ax = fanchart(datat['Minutos prepago salientes (miles)'],point_pred_pre,intv_pre,intv_pre,intv_pre)

plt.show()

### Simple Exponential Smoothing

### Líneas pospagas

In [ ]:
datat = datat.astype(float)
datat.dtypes

In [ ]:
pip install -U statsmodels


In [ ]:

from statsmodels.compat.python import string_types

import numpy as np
import pandas as pd
from scipy.optimize import basinhopping, brute, minimize
from scipy.spatial.distance import sqeuclidean
from scipy.special import inv_boxcox
from scipy.stats import boxcox

from statsmodels.base.model import Results
from statsmodels.base.wrapper import populate_wrapper, union_dicts, ResultsWrapper
from statsmodels.tsa.base.tsa_model import TimeSeriesModel
from statsmodels.tsa.tsatools import freq_to_period
import statsmodels.tsa._exponential_smoothers as smoothers



In [ ]:
model_ses = sm.tsa.SimpleExpSmoothing(datat['Minutos pospago salientes (miles)']).fit()
fitted_ses = model_ses.fittedvalues
model_ses.summary()


In [ ]:
mse_ses = sm.tools.eval_measures.mse(datat['Minutos pospago salientes (miles)'],fitted_ses)# para calcular el error medio cuadrado
print(' In-sample fit:')
print(' MSE               {0:.3f}'.format(mse_ses))

Generalmente no dan buenos resultados los criterios de bondad de ajuste como AIC y BIC, se deben calcular por fórmula

In [ ]:
N = len(datat['Minutos pospago salientes (miles)'])
loglik_ses=-(N/2)*(1+np.log(2*np.pi))-(N/2)*np.log(mse_ses)
aic_ses=-2*loglik_ses+2*3
bic_ses=-2*loglik_ses+np.log(N)*3
print(' Log-likelihood    %.3f' % (loglik_ses))
print(' AIC               {0:.3f}'.format(aic_ses))
print(' BIC               {0:.3f}'.format(bic_ses))

Son distintos los valores que arrojan y deben ser considerados ya que superan en +/- 2 puntos por lo tanto la selecciṕn entre modelos puede variar

In [ ]:
fig, ax= plt.subplots(figsize=(9,6))
datat['Minutos pospago salientes (miles)'].plot(color=red, label='Minutos pospago')
fitted_ses.plot(color='black', label='Simple exponential smoothing fit', alpha=0.8)
ax.set_xlabel('')
ax.set_ylabel('Inflation')
ax.set_title('Minutos pospago  salientes')
ax.set_xticks([], minor=True) 
plt.legend(loc='best')
sns.despine()
plt.show()

### Líneas Prepago

In [ ]:
model_sesr = sm.tsa.SimpleExpSmoothing(datat['Minutos prepago salientes (miles)']).fit()
fitted_sesr = model_sesr.fittedvalues
model_sesr.summary()


In [ ]:
mse_sesr = sm.tools.eval_measures.mse(datat['Minutos prepago salientes (miles)'],fitted_sesr)# para calcular el error medio cuadrado
print(' In-sample fit:')
print(' MSE               {0:.3f}'.format(mse_sesr))

El MSE es mucho más elevado para líneas de prepago

In [ ]:
N = len(datat['Minutos prepago salientes (miles)'])
loglik_sesr=-(N/2)*(1+np.log(2*np.pi))-(N/2)*np.log(mse_sesr)
aic_sesr=-2*loglik_sesr+2*3
bic_sesr=-2*loglik_sesr+np.log(N)*3
print(' Log-likelihood    %.3f' % (loglik_sesr))
print(' AIC               {0:.3f}'.format(aic_sesr))
print(' BIC               {0:.3f}'.format(bic_sesr))

In [ ]:
fig, ax= plt.subplots(figsize=(9,6))
datat['Minutos prepago salientes (miles)'].plot(color=red, label='Minutos prepago')
fitted_sesr.plot(color='black', label='Simple exponential smoothing fit', alpha=0.8)
ax.set_xlabel('')
ax.set_ylabel('Inflation')
ax.set_title('Minutos prepago  salientes')
ax.set_xticks([], minor=True) 
plt.legend(loc='best')
sns.despine()
plt.show()

La predicción de ses es mejor para líneas pospago

### Diagnóstico de los modelos ses

###  Líneas Pospago

In [ ]:
resid=datat['Minutos pospago salientes (miles)']-fitted_ses

In [ ]:
fig, ax= plt.subplots(figsize=(9,6))
resid.plot(color=blue)
ax.set_xlabel('')
ax.set_xticks([], minor=True) 
ax.set_title('Residual plot')
sns.despine()
plt.show()

 Encontramos que los residuos no están correlacionados, tienen una varianza bastante homogénea con un pico importante para el año 2017.

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
sm.graphics.tsa.plot_acf(resid, lags=20, ax=ax)
sns.despine()
plt.show()


In [ ]:
def hist(series):
    fig, ax= plt.subplots(figsize=(9,6))
    sns.distplot(series, ax=ax, hist_kws={'alpha': 0.8, 'edgecolor':'black', 'color': blue},  
                 kde_kws={'color': 'black', 'alpha': 0.7})
    sns.despine()
    return fig, ax

hist(resid)
plt.show()

Los residuos no poseen distribución gaussiana, los supuestos no se cumplen para el modelo

### Líneas Prepagas

In [ ]:
residr=datat['Minutos prepago salientes (miles)']-fitted_sesr

In [ ]:
fig, ax= plt.subplots(figsize=(9,6))
residr.plot(color=blue)
ax.set_xlabel('')
ax.set_xticks([], minor=True) 
ax.set_title('Residual plot')
sns.despine()
plt.show()

Varianza heterogénea 

In [ ]:
fig, ax = plt.subplots(figsize=(9,6))
sm.graphics.tsa.plot_acf(residr, lags=20, ax=ax)
sns.despine()
plt.show()


Hay que complementar los modelos ses, con ARIMA para determinar la estacionalidad, no queda claro en éstos modelos la existencia de tendencia y estacionalidad

In [ ]:
def hist(series):
    fig, ax= plt.subplots(figsize=(9,6))
    sns.distplot(series, ax=ax, hist_kws={'alpha': 0.8, 'edgecolor':'black', 'color': blue},  
                 kde_kws={'color': 'black', 'alpha': 0.7})
    sns.despine()
    return fig, ax

hist(residr)
plt.show()

Los residuos para líneas prepagas no presentan distribución gaussiana, habría que revisar la heterogeneidad de varianzas

### Validación del modelo

In [ ]:
# Real time forecasting - use it as a template - expanding window

validation=datat['Minutos pospago salientes (miles)']['2016Q1':].index # se valida el modelo con el data test que varía entre un "20%-50%"
start = datat['Minutos pospago salientes (miles)'].index.get_loc('2016Q1') 

pred1 = [] #random walk
pred2 = [] #SSE
actual= [] #valor real
for i in range(start, len(datat['Minutos pospago salientes (miles)'])):
    
    actual.append(datat['Minutos pospago salientes (miles)'].iloc[i]) # actual value
    
    pred1.append(datat['Minutos pospago salientes (miles)'].iloc[i-1]) # random walk forecast
    
    model = sm.tsa.SimpleExpSmoothing(datat['Minutos pospago salientes (miles)'].iloc[:i]).fit() 
    pred2.append(model.forecast(1)[0]) # SES forecast

columns=['RW', 'SES', 'Actual']
results_sm = np.vstack([pred1,pred2,actual]).T
results_sm = pd.DataFrame(results_sm, columns=columns, index=validation)
results_sm.tail()

EL MODELO DE Random Walk tiene mayor precisión en las predicciones para líneas pospagas

In [ ]:
from scipy import stats

table = pd.DataFrame(0.0, index=results_sm.columns[:-1], columns=['RMSE','SE'])
for i in range(2):
    table.iloc[i,0] = sm.tools.eval_measures.rmse(results_sm.iloc[:,i], results_sm.iloc[:,-1])
    table.iloc[i,1] = (results_sm.iloc[:,i] - results_sm.iloc[:,-1]).sem()
table.round(3)

El método RW presente menores errores en la predicción

### Líneas prepagas

In [ ]:
validation=datat['Minutos prepago salientes (miles)']['2016Q1':].index 
start = datat['Minutos prepago salientes (miles)'].index.get_loc('2016Q1') 

pred1 = [] #random walk
pred2 = [] #SSE
actual= [] #valor real
for i in range(start, len(datat['Minutos prepago salientes (miles)'])):
    
    actual.append(datat['Minutos prepago salientes (miles)'].iloc[i]) # actual value
    
    pred1.append(datat['Minutos prepago salientes (miles)'].iloc[i-1]) # random walk forecast
    
    model = sm.tsa.SimpleExpSmoothing(datat['Minutos prepago salientes (miles)'].iloc[:i]).fit() 
    pred2.append(model.forecast(1)[0]) # SES forecast

columns=['RW', 'SES', 'Actual']
results_sm = np.vstack([pred1,pred2,actual]).T
results_sm = pd.DataFrame(results_sm, columns=columns, index=validation)
results_sm.tail()

In [ ]:
table = pd.DataFrame(0.0, index=results_sm.columns[:-1], columns=['RMSE','SE'])
for i in range(2):
    table.iloc[i,0] = sm.tools.eval_measures.rmse(results_sm.iloc[:,i], results_sm.iloc[:,-1])
    table.iloc[i,1] = (results_sm.iloc[:,i] - results_sm.iloc[:,-1]).sem()
table.round(3)

### Predicciones

#### Expanding and rolling

### Líneas Pospago

In [ ]:
h=8
w=h*2
data = datat['Minutos pospago salientes (miles)']
model = sm.tsa.SimpleExpSmoothing
pred_index = datat['Minutos pospago salientes (miles)'].index[-1] + np.array(range(1,h+1))
pred_rolling = pd.Series(0,index=pred_index)
pred_expanding = pd.Series(0,index=pred_index)
orig_exp = None
orig_roll = None
for i in range(0,h):
    current_exp = pd.concat([data, pred_expanding.head(i)])
    expanding = model(current_exp).fit()
    pred_expanding.update(expanding.forecast(1))
    current_roll = pd.concat([data, pred_rolling.head(i)])
    rolling = model(current_roll.tail(w)).fit()  #en el rolling nos quedamos con un tail de w valores
    pred_rolling.update(rolling.forecast(1))
    if i==0:
        orig_fit = expanding.fittedvalues  #aca aplica lo de cross-validation de expanding window y rolling window
        orig_roll = rolling.fittedvalues
ax = data[-w:].plot(color='red',figsize=(20,12));
pd.concat([orig_fit[-w:],pred_expanding]).plot(ax=ax,color='blue',figsize=(20,12));
pd.concat([orig_roll[-w:],pred_rolling]).plot(ax=ax,color='green',figsize=(20,12));

Predicción con rolling y expanding para 64 años

### Líneas Prepago

In [ ]:
h=8
w=h*2
data = datat['Minutos prepago salientes (miles)']
model = sm.tsa.SimpleExpSmoothing
pred_index = datat['Minutos prepago salientes (miles)'].index[-1] + np.array(range(1,h+1))
pred_rolling = pd.Series(0,index=pred_index)
pred_expanding = pd.Series(0,index=pred_index)
orig_exp = None
orig_roll = None
for i in range(0,h):
    current_exp = pd.concat([data, pred_expanding.head(i)])
    expanding = model(current_exp).fit()
    pred_expanding.update(expanding.forecast(1))
    current_roll = pd.concat([data, pred_rolling.head(i)])
    rolling = model(current_roll.tail(w)).fit()  #en el rolling nos quedamos con un tail de w valores
    pred_rolling.update(rolling.forecast(1))
    if i==0:
        orig_fit = expanding.fittedvalues  #aca aplica lo de cross-validation de expanding window y rolling window
        orig_roll = rolling.fittedvalues
ax = data[-w:].plot(color='red',figsize=(20,12));
pd.concat([orig_fit[-w:],pred_expanding]).plot(ax=ax,color='blue',figsize=(20,12));
pd.concat([orig_roll[-w:],pred_rolling]).plot(ax=ax,color='green',figsize=(20,12));